## 1. 사용환경 준비
### 1.1 필요 라이브러리 설치
`pip install python-dotenv langchain langchain-openai faiss-cpu pypdf`

.env 파일에 OPENAI_API_KEY 를 지정해 준 뒤 사용함

In [7]:
from dotenv import (
    load_dotenv,
)  # dotenv 모듈 : .env 파일을 읽고 여기서 정의된 환경 변수를 시스템의 환경변수로 설정함
import os

from langchain_openai import ChatOpenAI # OpanAI 의 GPT 모델 사용 
from langchain_core.messages import (
    HumanMessage,
)  # Messages 작성(사용자의 요구사항이나 질문)을 위한 모듈 import

# 문서 로드 , 처리
from langchain.document_loaders import PyPDFLoader # PDF 문서 읽기 위한 모듈
from langchain.text_splitter import CharacterTextSplitter # 청킹 방법 1
from langchain.text_splitter import RecursiveCharacterTextSplitter # 청킹 방법 2

# 임벡딩 생성 및 벡터 스토어
from langchain_openai import OpenAIEmbeddings # OpenAI 를 사용해 임베딩 생성
import faiss # 벡터 검색을 위한 라이브러리
from langchain_community.vectorstores import FAISS # LangChain 과 FAISS 를 통합 -> 검색 가능 벡터 스토어 생성

# 프롬프트 정의, 실행 모듈
from langchain_core.prompts import ChatPromptTemplate # 프롬프트 템플릿 작성
from langchain_core.runnables import RunnablePassthrough # 데이터를 가공하지 않고 그대로 전달시 사용

# RAG 체인 구성
from langchain.chains import LLMChain # 9 번에서 LangChain 모델과 프롬프트를 연결한 RAG 체인 구성 위해 불러옴
from langchain.chains import RetrievalQA # Retriever 와 LLM 결합해 질문에 답하는 기능

In [8]:
"""from dotenv import (
    load_dotenv,
)  # dotenv 모듈 : .env 파일을 읽고 여기서 정의된 환경 변수를 시스템의 환경변수로 설정함
import os
"""

load_dotenv()  # .env 파일에서 환경 변수 로드

api_key = os.getenv(
    "OPENAI_API_KEY"
)  # os.getenv() : 환경변수에서 OPENAI_API_KEY 의 값 가져와 api_key 변수에 저장

## 2. 모델 로드하기

In [9]:
"""from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    HumanMessage,
)  # Messages 작성(사용자의 요구사항이나 질문)을 위한 모듈 import"""

# 모델 초기화 - 모델 선택
model = ChatOpenAI(model="gpt-4o-mini")

## 3. 문서 로드하기
`pip install pypdf`

PyPDFLoader : PDF 파일을 로드하고, 이를 문서로 변환. 이 문서들은 후속 작업에서 사용할 수 있도록 텍스트로 변환됨

In [10]:
"""from langchain.document_loaders import PyPDFLoader
"""

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader(
    "/Users/t2023-m0072/Desktop/assignment_LLM_RAG/PDF/인공지능산업최신동향_2024년11월호.pdf"
)

# 페이지 별 문서 로드
docs = loader.load()

# 로드된 문서 출력
"""for doc in docs:
    print(doc.page_content)"""
print(docs[5].page_content)

1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
3
유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간n유로폴의 보고서에 따르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오 분석 등에 활용되어 법 집행 업무를 대폭 개선할 수 있는 잠재력 보유n그러나 AI 도입을 위해서는 기술적 과제 해결 및 다양한 윤리적·사회적 이슈 대응이 필요하며, EU AI 법에 부합하도록 기존 AI 시스템에 대한 평가와 수정도 필요
KEY Contents
£유로폴, 법 집행에서 AI 기술의 윤리적이고 투명한 구현을 위한 고려사항 제시nEU 사법기관 유로폴(Europol)이 2024년 9월 24일 법 집행에서 효과적 범죄 퇴치를 위한 AI의 활용 가능성을 탐색한 보고서를 발간∙보고서는 법 집행에서 AI 기술을 윤리적이고 투명하게 구현하기 위한 지침 역할을 하며, AI의 이점과 과제를 함께 다룸으로써 법 집행에서 AI 사용 시 윤리적 고려 사항에 대한 인식 제고를 추구n보고서에 따르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오 분석, 생체인식 시스템 등에 활용되어 법 집행 업무를 대폭 개선할 수 있는 잠재력 보유∙법 집행기관은 AI 기반 데이터 분석을 활용해 범죄 활동에 대한 탐지와 대응 능력을 강화하고, AI 도구로 구조화되지 않은 데이터를 신속히 분석해 비상 상황의 의사결정을 위한 통찰력 확보 가능 ∙기계번역과 같은 AI 기반 도구는 여러 국가가 참여하는 조사에서 원활한 국제협력을 위해서도 필수적n그러나 법 집행에서 AI 도구의 효과적이고 책임 있는 활용을 위해 해결되어야 할 기술적 과제 및 다양한 윤리적·사회적 우려도 존재∙일례로 관할권 간 데이터 수집과 보관 관행의 차이에 따른 데이터셋의 편향으로 인해 AI 산출물의 무결성(無缺性)이 손상될 수 있어 표준화된 데이터 수집 규약 필요∙데이터 규모나 활용 사례의 복잡성과 관계없이 AI 도구를 효과적으로 사용하려면 다양한 데이터 규모와 운영 요구사항에 적응할 수 있는 확장성

## 4. 문서 청크로 나누기

### 4.0 Chunk VS Token
#### Chunk
: 문자나 단어 등의 더 큰 의미 단위로 텍스트를 나누는 방식. 문맥적 의미를 기준으로 나눈다
* 특징
  * 크기 : 문자수 (characters) 나 단어수 (words) 로 정의됨
  * 의미 단위 : 청크는 텍스트를 의미가 있는 단위로 나누는데 중점을 둠. 
    * 문장 단위
    * 문단 단위
    * 구문 단위 등

#### Token
: 텍스트의 기본적인 처리단위. NLP 모델에서 텍스트를 벡터화 하거나 분석하기 전에 텍스트를 나누는 가장 작은 단위. 모델이 이해할 수 있는 기본적인 단위
* 토큰화 (Tokenization) : 텍스트를 단어, 구두점, 특수 문자 등으로 분할하는 과정
* 특징
  * 크기 : 고정되어 있지 않음. 단어, 하위 단어, 구두점 등 다양한 요소로 구성됨
  * 언어 모델의 기본 단위 : GPT 같은 언어 모델은 토큰을 입력 받아 처리함

## 4.1 CharacterTextSplitter
Langchain 에서 제공하는 문서 분할 도구. 문자 단위로 텍스트를 나눔.
### 4.1.1 CharacterTextSplitter 특징 
* **고정된 청크 크기** : 텍스트를 동일한 크기로 나눔. 일관성 있는 처리에 유용
* **속도** : 단순한 방식 덕분에 처리 속도가 빠르고, 대규모 데이터셋을 처리할 때 효율적
* **단순성** : 구현이 간단. 복잡한 설정 없이 다양한 응용 프로그램에 쉽게 통합
* 중복 오버랩 (Overlap) : 각 청크 간 겹치는 부분을 두어 텍스트를 자연스럽게 나눌 수 있음. 중복을 추가해 나눠진 청크가 더 의미 있는 단위로 나뉘어 모델이 문맥을 더 잘 이해할 수 있음


### 4.1.2 parameter
* `separator` : 텍스트를 나누는 기준을 설정
  * 기본값 : 공백 `' '`
  * 줄 단위 나누기 : `\n`
  * 문장 단위 나누기 : `.`
---
* `chunk_size` : 각 분할된 청크의 최대 길이를 설정
  * `chunk_size=100` : 각 청크가 최대 100자 까지 포함됨을 의미
  * 일반적으로 512~1000 토큰
---
* `chunk_overlap` : 청크 간 곂치는 문자 수를 설정  -> 각 텍스트간 의미가 끊어지지 않도록 하기 위함
  * `chunk_overlap=10` : 각 청크가 이전 청크와 최대 10자 겹침
  * 보통 `chunk_size` 의 10~20% 사용
  * 중복이 너무 크면 처리 효율 하락. 적절한 설정 필요
---
* `length_function` : 각 청크의 길이 계산 함수
  * `length_function=len` : 각 청크의 길이가 문자 수로 계산됨. 
  * `length_function=lambda text: len(text.split())` : 단어 수로 텍스트 나눔
---
* `is_separator_regex` : separator 가 정규표현식 인지
  * `is_separator_regex=False`(기본값) : separator로 지정된 텍스트가 정확히 일치하는 부분에서 텍스트를 나눔
  *  `is_separator_regex=True` : separator 에 정규 표현식을 사용할 수 있음
     *  `separator=r'\n+'`를 사용하면 여러 줄 바꿈이 있더라도 이를 하나의 분할 기준으로 사용

In [11]:
"""from langchain.text_splitter import CharacterTextSplitter
"""

text_splitter = CharacterTextSplitter(
    separator="\n\n",  # 분할기준 : 두 줄 바꿈을 기준으로 텍스트 나누기
    chunk_size=500,  # 청크 크기 : 최대 100자
    chunk_overlap=50,  # 중복 오버랩 : 각 청크가 최대 10자 겹침
    length_function=len,  # 길이 계산 함수 : 문자 수 기준 (len())
    is_separator_regex=False,  # 구분자가 정규 표현식인지 여부 (False : 정확히 일치하는 구분자 기준)
)

splits = text_splitter.split_documents(docs)

# 청킹된 내용 상위 10개 출력
top_10 = splits[:10]
# print(f"Chunk 3:{splits[3]}")
for i, chunk in enumerate(top_10, 1):
    page_content = chunk.page_content
    print(f"Chunk {i}:{page_content[:300]}\n\n")
# print(f"splits 길이 : {len(splits)}")

Chunk 1:2024년 11월호


Chunk 2:2024년 11월호
Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1   ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간··············································3   ▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 


Chunk 3:Ⅰ. 인공지능 산업 동향 브리프


Chunk 4:1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
1
미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석n미국 민권위원회에 따르면 연방정부와 법 집행기관에서 얼굴인식 기술이 빠르게 도입되고 있으나 이를 관리할 지침과 감독의 부재로 민권 문제를 초래할 위험 존재n미국 민권위원회는 연방정부의 책임 있는 얼굴인식 기술 사용을 위해 운영 프로토콜 개발과 실제 사용 상황의 얼굴인식 기술 평가 및 불평등 완화, 지역사회의 의견 수렴 등을 권고
KEY Contents
£연방정부의 얼굴인식 기술 도입에 대한 


Chunk 5:SPRi AI Brief |  2024-11월호
2
미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표n미국 백악관 예산관리국이 바이든 대통령의 AI 행정명령에 따라 연방정부의 책임 있는 AI 조달을 지원하기 위한 지침을 발표 n지침은 정부 기관의 AI 조달 시 AI의 위험과 성과를 관리할 수 있는 모범 관행의 수립 및 최상의 AI 솔루션을 사용하기 위한 공급업체 시장의 경쟁 보장, 정부 기관 간 협업을 요구  
KEY Contents
£백악관 예산관리국, 연방정부의 AI 조달 시 책임성을 증진하기 위한 모범 관


Chunk 6:1. 정책/법제  

## 4.2 RecursiveCharacterTextSplitter
내용의 **문맥을 유지**하면서 텍스트를 관리 가능한 청크로 분할 하도록 하는 청킹 방식. 대량의 텍스트를 처리할 때 유용. 관련 정보들이 서로 인접하게 유지되므로 **가독성**과 **이해도**를 높이는데 효과적.
### 4.2.1 RecursiveCharacterTextSplitter 특징
* **계층적 세분화** : 텍스트를 문맥에 나누어 의미가 각 청크 간에 보존됨
* **적응형 청크 크기** : 텍스트 내용에 따라 청크 크기가 달라질 수 있음. 길이가 다른 문단이 포함된 문서에 유용
* **문맥 보존** : 주변 텍스트를 고려해 청킹 과정 중 중요한 정보가 손실되는 것을 최소화 함

In [12]:
"""from langchain.text_splitter import RecursiveCharacterTextSplitter
"""

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 각 청크 최대 100자 까지
    chunk_overlap=15,  # 청크간 최대 10자 중복
    length_function=len,  # 길이 계산 함수 : 문자 수 기준 (len())
    is_separator_regex=False,  # 구분자가 정규 표현식인지 여부 (False : 정확히 일치하는 구분자 기준)
)

splits = recursive_text_splitter.split_documents(docs)

# 청킹된 내용의 상위 10개 출력
top_10 = splits[:10]
# print(f"Chunk 3:{splits[3]}")
for i, chunk in enumerate(top_10, 1):
    page_content = chunk.page_content
    print(f"Chunk {i}:{page_content[:300]}\n\n")

Chunk 1:2024년 11월호


Chunk 2:2024년 11월호


Chunk 3:Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1   ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간··············································3   ▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표·········


Chunk 4:31%가 AI 스타트업에 집중··············6   ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개···································································7   ▹ 메타, 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개···························8   ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o 능가하는 성능의 오픈소스 LLM ‘몰모’ 공개····9   ▹ 미스트랄AI, 온디바이스용 AI 모델 ‘레 미니스


Chunk 5:AI 관련 연구자들이 수상············································12   ▹ 미국 국무부, AI 연구에서 국제협력을 위한 ‘글로벌 AI 연구 의제’ 발표························13   ▹ 일본 AI안전연구소, AI 안전성에 대한 평가 관점 가이드 발간········································14   ▹ 구글 딥마인드, 반도체 칩 레이아웃 설계하는 AI 모델 ‘알파칩’ 발표·····························15   ▹ AI21 


Chunk 6:2025년 중 이직 고려····

## 5. 벡터 임베딩 생성

In [13]:
"""from langchain_openai import OpenAIEmbeddings
"""

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

## 6. 벡터 스토어 생성

### FAISS 
(Facebook AI Similarity Search)
* Vector DB 
* `FAISS.from_documents()` : 문서 집합을 벡터화하여 벡터 스토어(검색 가능한 Vector DB) 생성 
* `.from_documents()` : 문서들의 집합을 받아서 해당 문서들을 벡터로 변환. 변환된 벡터를 FAISS 인덱스로 저장
  * `documents` : 문서들 
  * `embedding` : 임베딩 모델

In [14]:
"""import faiss
from langchain_community.vectorstores import FAISS"""
# 4.2 에서 RecursiveCharacterTextSplitter 로 청킹 했던 문서 (splits) 를 5에서 생성한 OpenAI 의 임베딩 모델을 사용해 벡터로 변환 후 저장
vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

## 7. FAISS 를 Retriever 로 변환

#### Retriever 
* 유사도 기반 검색을 수행
* 벡터 스토어에서 문서를 검색하려면 이와 같은 객체가 필요함. 

In [15]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})
# search_kwargs={"k":1} : 검색할 결과의 개수로, 가장 유사한 문서 1개 검색

## 8. 프롬프트 템플릿 정의

In [16]:
"""from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough"""

'''# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages(   # .from_messages() : 메시지의 리스트를 기반으로 템플릿 생성
    [
        (
            "system", # 시스템 메시지로 모델의 행동 설정
            """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Answer in Korean.""",
        ),
        ("user", "Context: {context}\\n\\nQuestion: {question}"), # 사용자 메시지로 모델에게 주어질 정보를 제공
    ] # {context} : 실제 문맥이 들어감, {question} : 사용자가 던진 질문이 들어감
)'''

'# 프롬프트 템플릿 정의\ncontextual_prompt = ChatPromptTemplate.from_messages(   # .from_messages() : 메시지의 리스트를 기반으로 템플릿 생성\n    [\n        (\n            "system", # 시스템 메시지로 모델의 행동 설정\n            """You are an assistant for question-answering tasks.\n            Use the following pieces of retrieved context to answer the question.\n            If you don\'t know the answer, just say that you don\'t know.\n            Answer in Korean.""",\n        ),\n        ("user", "Context: {context}\\n\\nQuestion: {question}"), # 사용자 메시지로 모델에게 주어질 정보를 제공\n    ] # {context} : 실제 문맥이 들어감, {question} : 사용자가 던진 질문이 들어감\n)'

### 도전과제를 위한 프롬프트 불러오기

In [19]:
# Prompts 폴더 내 프롬프트 파일 경로
PROMPT_FILES = {
    "prompt1": "/Users/t2023-m0072/Desktop/assignment_LLM_RAG/Prompts/prompts1.txt",
    "prompt2": "/Users/t2023-m0072/Desktop/assignment_LLM_RAG/Prompts/prompts2.txt",
    "prompt3": "/Users/t2023-m0072/Desktop/assignment_LLM_RAG/Prompts/prompts3.txt",
}

def load_prompt(file_path):
    """
    주어진 파일 경로에서 프롬프트 텍스트를 읽어오는 함수.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Prompt file not found: {file_path}")
    with open(file_path, "r", encoding="utf-8") as f:
        prompt_text = f.read()
    return prompt_text


# 프롬프트를 선택하여 로드
def select_prompt():
    print("Available prompts:")
    for key in PROMPT_FILES.keys():
        print(f" - {key}")
    selected = input("프롬프트를 선택하세요 (예시 : prompt1): ").strip()
    if selected not in PROMPT_FILES:
        print("Invalid prompt selection. Using default: 'prompt1'")
        selected = "prompt1"
    return load_prompt(PROMPT_FILES[selected])

# 프롬프트 로드 및 정의
prompt_text = select_prompt()

# ChatPromptTemplate 생성
contextual_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            prompt_text,  # 파일에서 로드한 프롬프트 적용
        ),
        (
            "user",
            "Context: {context}\\n\\nQuestion: {question}",
        ),
    ]
)


Available prompts:
 - prompt1
 - prompt2
 - prompt3


## 9. RAG 체인 구성

In [22]:
'''from langchain.chains import LLMChain
'''
# 입력값 그대로 반환하는 클래스 정의
class SimplePassThrough:  
    def invoke(self, inputs, **kwargs):
        return inputs
        
# llm_chain 설정
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)
# llm=model : 사용할 언어 모델 지정 ('gpt-4o-mini')
# prompt : 사용할 프롬프트 템플릿 지정


# 문서 내용을 프롬프트 템플릿에 맞게 변환하는 역할하는 클래스 정의
class ContextToPrompt: 
    def __init__(self, prompt_template): # 클래스 인스턴스 생성시 사용될 프롬프트 템플릿 전달 받음
        self.prompt_template = prompt_template

    def invoke(self, inputs): 
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs]) 
        else:
            context_text = inputs

        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text, question=inputs.get("question", "")
        )
        return formatted_prompt


# Retriever를 invoke() 메서드로 래핑하는 클래스 정의 -> 문서 검색 수행
class RetrieverWrapper:
    def __init__(self, retriever): # retriever : 문서 검색 수행 객체
        self.retriever = retriever

    def invoke(self, inputs): 
        if isinstance(inputs, dict):
            query = inputs.get("question", "") # inputs 에서 question 추출
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs


# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever), # 검색된 문서들 가져옴. retriever (실제 검색 작업 수행)
    "prompt": ContextToPrompt(contextual_prompt), # 문서 내용을 프롬프트 템플릿에 맞게 변환
    "llm": model, # 언어 모델 설정
}

## 10. 챗봇 구동 확인

In [21]:
while True:
    print("-------------------------------------")
    query = input("질문을 입력하세요 ('exit' 입력시 종료) >")
    print("질문 >")
    print(query)
    # 종료 조건 확인 (exit 입력 시 종료)
    if query.lower() == "exit":  # .lower() 메서드 사용
        print("챗봇을 종료합니다!")
        break

    # Step 1: 'context'에서 답변을 받아오기
    # 'context'가 RunnableSequence라면 invoke()를 사용하여 결과를 받아옴
    response_context = rag_chain_debug["context"].invoke({"question": query})

    # Step 2: prompt를 생성하는 단계
    # 'prompt'가 RunnableSequence라면 invoke()를 사용하여 결과를 받아옴
    prompt_msg = rag_chain_debug["prompt"].invoke(
        {"context": response_context, "question": query}
    )

    # Step 3: LLM을 이용하여 답변 생성
    # 'llm'이 RunnableSequence라면 invoke()를 사용하여 최종 답변을 생성
    response_llm = rag_chain_debug["llm"].invoke(prompt_msg)  # .invoke() 사용

    # Step 4: 답변 출력
    print("\n답변 >")
    print(
        response_llm.content
    )  # 또는 response_llm.content, 정확한 속성은 모델에 따라 다름

    try:
        response_context = rag_chain_debug["context"].invoke({"question": query})
    except Exception as e:
        print(f"Error in context invocation: {e}")
        continue  # 오류가 발생하면 다시 질문을 받도록 함

-------------------------------------
질문 >


답변 >
질문이 없기 때문에 답변을 드릴 수 없습니다. 추가 질문이 있으시면 알려주세요.
-------------------------------------
